In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

Using TensorFlow backend.


In [2]:
gan = GAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

In [3]:
gan.train(epochs=30000, batch_size=32, sample_interval=200)

/Users/davidfoster/.virtualenvs/gdl-kerasvae/lib/python3.6/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.946417, acc.: 29.69%] [G loss: 0.672101]
1 [D loss: 0.527054, acc.: 64.06%] [G loss: 0.665918]
2 [D loss: 0.373032, acc.: 79.69%] [G loss: 0.803168]
3 [D loss: 0.340927, acc.: 76.56%] [G loss: 0.906053]
4 [D loss: 0.278831, acc.: 93.75%] [G loss: 1.050505]
5 [D loss: 0.217311, acc.: 98.44%] [G loss: 1.202190]
6 [D loss: 0.205817, acc.: 100.00%] [G loss: 1.277849]
7 [D loss: 0.168885, acc.: 100.00%] [G loss: 1.344582]
8 [D loss: 0.152928, acc.: 100.00%] [G loss: 1.439111]
9 [D loss: 0.154437, acc.: 100.00%] [G loss: 1.552042]
10 [D loss: 0.141943, acc.: 100.00%] [G loss: 1.642107]
11 [D loss: 0.118015, acc.: 100.00%] [G loss: 1.728682]
12 [D loss: 0.108579, acc.: 100.00%] [G loss: 1.782911]
13 [D loss: 0.103542, acc.: 100.00%] [G loss: 1.900077]
14 [D loss: 0.093571, acc.: 100.00%] [G loss: 2.000989]
15 [D loss: 0.084054, acc.: 100.00%] [G loss: 2.018569]
16 [D loss: 0.091587, acc.: 100.00%] [G loss: 2.138663]
17 [D loss: 0.080455, acc.: 100.00%] [G loss: 2.152542]
18 [D lo

152 [D loss: 0.194958, acc.: 95.31%] [G loss: 3.082331]
153 [D loss: 0.143593, acc.: 98.44%] [G loss: 2.770835]
154 [D loss: 0.109502, acc.: 96.88%] [G loss: 2.928789]
155 [D loss: 0.085682, acc.: 96.88%] [G loss: 3.444491]
156 [D loss: 0.157968, acc.: 96.88%] [G loss: 2.477961]
157 [D loss: 0.171417, acc.: 93.75%] [G loss: 3.308766]
158 [D loss: 0.105182, acc.: 100.00%] [G loss: 3.725972]
159 [D loss: 0.327251, acc.: 84.38%] [G loss: 2.873170]
160 [D loss: 0.105133, acc.: 95.31%] [G loss: 4.093606]
161 [D loss: 0.438167, acc.: 75.00%] [G loss: 2.997654]
162 [D loss: 0.092220, acc.: 96.88%] [G loss: 3.733395]
163 [D loss: 0.234477, acc.: 89.06%] [G loss: 2.914204]
164 [D loss: 0.114268, acc.: 96.88%] [G loss: 3.739561]
165 [D loss: 0.228263, acc.: 90.62%] [G loss: 3.198082]
166 [D loss: 0.079764, acc.: 98.44%] [G loss: 3.475519]
167 [D loss: 0.144534, acc.: 95.31%] [G loss: 2.793739]
168 [D loss: 0.193390, acc.: 90.62%] [G loss: 3.891973]
169 [D loss: 0.351040, acc.: 84.38%] [G loss: 3

304 [D loss: 0.669668, acc.: 43.75%] [G loss: 0.708181]
305 [D loss: 0.714306, acc.: 42.19%] [G loss: 0.698607]
306 [D loss: 0.706662, acc.: 39.06%] [G loss: 0.679835]
307 [D loss: 0.673029, acc.: 50.00%] [G loss: 0.699246]
308 [D loss: 0.686410, acc.: 40.62%] [G loss: 0.719963]
309 [D loss: 0.720547, acc.: 40.62%] [G loss: 0.676939]
310 [D loss: 0.707559, acc.: 42.19%] [G loss: 0.681684]
311 [D loss: 0.670236, acc.: 46.88%] [G loss: 0.680781]
312 [D loss: 0.673035, acc.: 40.62%] [G loss: 0.686902]
313 [D loss: 0.688701, acc.: 43.75%] [G loss: 0.666313]
314 [D loss: 0.700164, acc.: 40.62%] [G loss: 0.662022]
315 [D loss: 0.689725, acc.: 45.31%] [G loss: 0.684106]
316 [D loss: 0.715316, acc.: 37.50%] [G loss: 0.673064]
317 [D loss: 0.675380, acc.: 43.75%] [G loss: 0.722800]
318 [D loss: 0.663071, acc.: 48.44%] [G loss: 0.702304]
319 [D loss: 0.688822, acc.: 39.06%] [G loss: 0.695159]
320 [D loss: 0.693586, acc.: 45.31%] [G loss: 0.682579]
321 [D loss: 0.706292, acc.: 39.06%] [G loss: 0.

456 [D loss: 0.726473, acc.: 51.56%] [G loss: 0.670405]
457 [D loss: 0.653074, acc.: 54.69%] [G loss: 0.722769]
458 [D loss: 0.666417, acc.: 54.69%] [G loss: 0.717933]
459 [D loss: 0.677634, acc.: 53.12%] [G loss: 0.764504]
460 [D loss: 0.686172, acc.: 45.31%] [G loss: 0.754578]
461 [D loss: 0.680429, acc.: 43.75%] [G loss: 0.711422]
462 [D loss: 0.664013, acc.: 64.06%] [G loss: 0.714841]
463 [D loss: 0.640819, acc.: 59.38%] [G loss: 0.726354]
464 [D loss: 0.650814, acc.: 57.81%] [G loss: 0.731548]
465 [D loss: 0.674174, acc.: 54.69%] [G loss: 0.721849]
466 [D loss: 0.656242, acc.: 56.25%] [G loss: 0.705322]
467 [D loss: 0.639720, acc.: 64.06%] [G loss: 0.695790]
468 [D loss: 0.629959, acc.: 68.75%] [G loss: 0.723835]
469 [D loss: 0.648523, acc.: 65.62%] [G loss: 0.727895]
470 [D loss: 0.653908, acc.: 68.75%] [G loss: 0.735668]
471 [D loss: 0.686791, acc.: 53.12%] [G loss: 0.771358]
472 [D loss: 0.668463, acc.: 57.81%] [G loss: 0.787033]
473 [D loss: 0.677382, acc.: 46.88%] [G loss: 0.

...

29362 [D loss: 0.571693, acc.: 68.75%] [G loss: 0.879785]
29363 [D loss: 0.689208, acc.: 57.81%] [G loss: 1.007096]
29364 [D loss: 0.604495, acc.: 60.94%] [G loss: 1.160587]
29365 [D loss: 0.546098, acc.: 70.31%] [G loss: 1.195736]
29366 [D loss: 0.647330, acc.: 65.62%] [G loss: 1.095852]
29367 [D loss: 0.500495, acc.: 84.38%] [G loss: 1.128832]
29368 [D loss: 0.632883, acc.: 64.06%] [G loss: 1.029183]
29369 [D loss: 0.604675, acc.: 68.75%] [G loss: 1.028574]
29370 [D loss: 0.651891, acc.: 57.81%] [G loss: 1.070410]
29371 [D loss: 0.610232, acc.: 64.06%] [G loss: 1.137967]
29372 [D loss: 0.692236, acc.: 59.38%] [G loss: 1.075836]
29373 [D loss: 0.653271, acc.: 64.06%] [G loss: 0.978309]
29374 [D loss: 0.651457, acc.: 57.81%] [G loss: 1.013651]
29375 [D loss: 0.636312, acc.: 64.06%] [G loss: 1.179670]
29376 [D loss: 0.638503, acc.: 60.94%] [G loss: 1.003631]
29377 [D loss: 0.644953, acc.: 64.06%] [G loss: 1.019537]
29378 [D loss: 0.634226, acc.: 68.75%] [G loss: 1.052826]
29379 [D loss:

29507 [D loss: 0.681334, acc.: 57.81%] [G loss: 0.920988]
29508 [D loss: 0.634607, acc.: 70.31%] [G loss: 0.916729]
29509 [D loss: 0.686308, acc.: 57.81%] [G loss: 1.069427]
29510 [D loss: 0.771275, acc.: 53.12%] [G loss: 0.990061]
29511 [D loss: 0.729648, acc.: 57.81%] [G loss: 1.114261]
29512 [D loss: 0.600558, acc.: 73.44%] [G loss: 0.993536]
29513 [D loss: 0.637435, acc.: 68.75%] [G loss: 1.015221]
29514 [D loss: 0.628696, acc.: 64.06%] [G loss: 1.022086]
29515 [D loss: 0.601830, acc.: 64.06%] [G loss: 1.113715]
29516 [D loss: 0.595185, acc.: 64.06%] [G loss: 1.109833]
29517 [D loss: 0.640104, acc.: 59.38%] [G loss: 1.112124]
29518 [D loss: 0.623484, acc.: 60.94%] [G loss: 0.932566]
29519 [D loss: 0.610097, acc.: 67.19%] [G loss: 1.100468]
29520 [D loss: 0.583698, acc.: 73.44%] [G loss: 1.046873]
29521 [D loss: 0.568294, acc.: 73.44%] [G loss: 1.191779]
29522 [D loss: 0.577530, acc.: 67.19%] [G loss: 1.081997]
29523 [D loss: 0.607456, acc.: 67.19%] [G loss: 0.981416]
29524 [D loss:

29654 [D loss: 0.562811, acc.: 75.00%] [G loss: 0.967960]
29655 [D loss: 0.661399, acc.: 59.38%] [G loss: 1.061342]
29656 [D loss: 0.667954, acc.: 60.94%] [G loss: 0.913647]
29657 [D loss: 0.595601, acc.: 71.88%] [G loss: 1.102653]
29658 [D loss: 0.679713, acc.: 57.81%] [G loss: 1.063631]
29659 [D loss: 0.649037, acc.: 64.06%] [G loss: 0.985037]
29660 [D loss: 0.638835, acc.: 65.62%] [G loss: 0.991373]
29661 [D loss: 0.655191, acc.: 57.81%] [G loss: 0.881807]
29662 [D loss: 0.690785, acc.: 57.81%] [G loss: 1.154664]
29663 [D loss: 0.558835, acc.: 70.31%] [G loss: 0.990212]
29664 [D loss: 0.571350, acc.: 73.44%] [G loss: 0.975315]
29665 [D loss: 0.656961, acc.: 65.62%] [G loss: 0.866610]
29666 [D loss: 0.631547, acc.: 64.06%] [G loss: 1.014405]
29667 [D loss: 0.671646, acc.: 53.12%] [G loss: 0.935831]
29668 [D loss: 0.604078, acc.: 75.00%] [G loss: 0.972362]
29669 [D loss: 0.587092, acc.: 70.31%] [G loss: 0.959029]
29670 [D loss: 0.535689, acc.: 76.56%] [G loss: 0.983013]
29671 [D loss:

29799 [D loss: 0.575868, acc.: 68.75%] [G loss: 1.050034]
29800 [D loss: 0.639974, acc.: 57.81%] [G loss: 1.012012]
29801 [D loss: 0.565652, acc.: 71.88%] [G loss: 0.913226]
29802 [D loss: 0.628486, acc.: 67.19%] [G loss: 1.082454]
29803 [D loss: 0.620786, acc.: 64.06%] [G loss: 0.958111]
29804 [D loss: 0.636842, acc.: 60.94%] [G loss: 0.910924]
29805 [D loss: 0.584502, acc.: 71.88%] [G loss: 1.001493]
29806 [D loss: 0.616408, acc.: 70.31%] [G loss: 0.898943]
29807 [D loss: 0.644411, acc.: 59.38%] [G loss: 0.910334]
29808 [D loss: 0.660041, acc.: 64.06%] [G loss: 1.089236]
29809 [D loss: 0.693476, acc.: 59.38%] [G loss: 0.971027]
29810 [D loss: 0.611036, acc.: 56.25%] [G loss: 0.968083]
29811 [D loss: 0.676310, acc.: 57.81%] [G loss: 1.107181]
29812 [D loss: 0.711940, acc.: 57.81%] [G loss: 1.002661]
29813 [D loss: 0.655270, acc.: 62.50%] [G loss: 0.975113]
29814 [D loss: 0.607835, acc.: 68.75%] [G loss: 0.996438]
29815 [D loss: 0.587701, acc.: 70.31%] [G loss: 1.044662]
29816 [D loss:

29942 [D loss: 0.650785, acc.: 57.81%] [G loss: 1.062200]
29943 [D loss: 0.637929, acc.: 68.75%] [G loss: 1.076586]
29944 [D loss: 0.585105, acc.: 68.75%] [G loss: 1.167587]
29945 [D loss: 0.655073, acc.: 64.06%] [G loss: 1.027035]
29946 [D loss: 0.676240, acc.: 53.12%] [G loss: 0.970116]
29947 [D loss: 0.554903, acc.: 78.12%] [G loss: 1.004914]
29948 [D loss: 0.670506, acc.: 60.94%] [G loss: 0.908490]
29949 [D loss: 0.577023, acc.: 68.75%] [G loss: 0.964078]
29950 [D loss: 0.710099, acc.: 62.50%] [G loss: 0.898538]
29951 [D loss: 0.541257, acc.: 76.56%] [G loss: 1.021400]
29952 [D loss: 0.550173, acc.: 79.69%] [G loss: 1.031550]
29953 [D loss: 0.624199, acc.: 64.06%] [G loss: 0.955495]
29954 [D loss: 0.651731, acc.: 59.38%] [G loss: 1.007957]
29955 [D loss: 0.679042, acc.: 56.25%] [G loss: 1.080739]
29956 [D loss: 0.595116, acc.: 70.31%] [G loss: 1.037755]
29957 [D loss: 0.669347, acc.: 60.94%] [G loss: 0.988642]
29958 [D loss: 0.586710, acc.: 70.31%] [G loss: 0.977268]
29959 [D loss: